In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
#convertir texto en un valor a una matrix, cada columna es una palabra
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import unicodedata
import pickle
import re
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import metrics

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [5]:
df.sample(10)

,url,is_spam
2277,https://www.theskimm.com/skimmpicks,True
2624,https://www.politico.com/news/2020/06/30/democ...,False
248,https://www.tulsaworld.com/news/trump-in-tulsa...,False
2264,https://www.nytimes.com/2020/06/29/world/asia/...,False
2485,https://bigspaceship.us9.list-manage.com/unsub...,True
2879,https://en.wikipedia.org/wiki/Charles_Goodhart,False
1970,https://www.vox.com/2020/6/27/21304910/elijah-...,False
1476,https://austinkleon.com/steal/,True
786,https://qz.com/guides/,False
2159,https://subscribe.washingtonpost.com/newslette...,True


In [6]:
df = df.drop_duplicates().reset_index(drop = True)

In [7]:
print("spam count: " +str(len(df.loc[df.is_spam=='True'])))
print("not spam count: " +str(len(df.loc[df.is_spam=='True'])))
print(df.shape)
df['is_spam'] = df['is_spam'].astype(int)

spam count: 0
not spam count: 0
(2369, 2)


In [8]:
#Eliminamos los caracteres desconocidos
df['url']=df['url'].str.strip()
df['url']=df['url'].str.lower()
df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1
...,...,...
2364,https://www.theverge.com/2020/6/29/21306889/di...,0
2365,https://www.smartcitiesworld.net/news/news/dee...,0
2366,https://techcrunch.com/2019/07/04/an-optimisti...,0
2367,https://www.technologyreview.com/2019/12/20/13...,0


In [9]:
df['url']=df['url'].str.replace('!','')
df['url']=df['url'].str.replace(',','')
df['url']=df['url'].str.normalize('NFKC') #ANALIZA TEXTO NO LATINO Y LO TRATA DE ARREGLAR
df['url']=df['url'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)
df.sample(10)

,url,is_spam
1189,https://w.creativemornings.com/,1
574,https://w.firstaidbeauty.com/fab-aid,0
1053,https://simplisafe.com/value,1
330,https://w.morningbrew.com/daily/stories/2020/0...,0
1187,https://creativemornings.us2.list-manage.com/p...,1
1738,https://w.dmagazine.com/publications/d-magazin...,0
913,https://w.cnn.com/2020/06/25/media/splash-moun...,0
1057,https://w.avclub.com/why-are-madonna-and-the-g...,0
1681,https://w.reuters.com/article/us-health-corona...,0
1993,https://w.morningbrew.com/daily/stories/2020/0...,0


In [10]:
def normalize_string(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else: 
        result=None
    return result

df['url']=df['url'].apply(normalize_string)
df.sample(10)

,url,is_spam
1377,https://w.lelo.com/pride-month,0
108,https://w.mentalfloss.com/article/625712/got-1...,0
2255,https://numlock.substack.com/p/numlock-sunday-...,1
1731,https://w.cnbc.com/2020/06/26/this-chart-shows...,0
1145,https://w.tiktok.com/@kcdadventure,0
1144,https://w.tiktok.com/@evanatoredits,0
1375,https://digg.com/2020/the-oldest-restaurants-i...,0
953,https://w.youtube.com/watch?v=q0hyywkxf0q,0
2345,https://w.marketwatch.com/story/f40a431c-bafa-...,0
740,https://w.cnbc.com/2020/06/24/new-ford-f-150-t...,0


In [13]:
def url(text):
    return re.sub(r'(https://www|https://)', '', text)

In [15]:
def comas(text):
    """
    Elimina comas del texto
    """
    return re.sub(',', ' ', text)

def espacios(text):
    """
    Elimina enters dobles por un solo enter
    """
    return re.sub(r'(\n{2,})','\n', text)

def minuscula(text):
    """
    Cambia mayusculas a minusculas
    """
    return text.lower()

def numeros(text):
    """
    Sustituye los numeros
    """
    return re.sub('([\d]+)', ' ', text)

def caracteres_no_alfanumericos(text):
    """
    Sustituye caracteres raros, no digitos y letras
    Ej. hola 'pepito' como le va? -> hola pepito como le va
    """
    return re.sub("(\\W)+"," ",text)

def comillas(text):
    """
    Sustituye comillas por un espacio
    Ej. hola 'pepito' como le va? -> hola pepito como le va?
    """
    return re.sub("'"," ", text)

def palabras_repetidas(text):
    """
    Sustituye palabras repetidas

    Ej. hola hola, como les va? a a ustedes -> hola, como les va? a ustedes
    """
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

def esp_multiple(text):
    """
    Sustituye los espacios dobles entre palabras
    """
    return re.sub(' +', ' ',text)

In [16]:
df['url_limpia'] = df['url'].apply(url).apply(caracteres_no_alfanumericos).apply(esp_multiple)

In [17]:
df.sample(4)

,url,is_spam,url_limpia
964,https://digg.cmail20.com/t/t-u-ptdjtuk-vkjjhbl...,0,digg cmail20 com t t u ptdjtuk vkjjhbly ik
118,https://w.ft.com/content/90d666a2-621e-487e-80...,0,w ft com content 90d666a2 621e 487e 8048 e86c0...
9,https://w.theguardian.com/travel/2020/jun/18/e...,0,w theguardian com travel 2020 jun 18 end of to...
1372,https://w.vox.com/culture/2020/6/26/21291757/b...,0,w vox com culture 2020 6 26 21291757 best movi...


In [18]:
vec = CountVectorizer().fit_transform(df['url_limpia'])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(vec, df['is_spam'], stratify = df['is_spam'], random_state = 207)

In [23]:
classifier = SVC(C = 1.0, kernel = 'linear', gamma = 'auto')

In [24]:
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       532
           1       0.73      0.67      0.70        61

    accuracy                           0.94       593
   macro avg       0.85      0.82      0.83       593
weighted avg       0.94      0.94      0.94       593



In [27]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

grid = GridSearchCV(SVC(random_state=1234),param_grid,verbose=2)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   0.3s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   0.1s
[CV] END .....................C=0.1, gamma=1, k

GridSearchCV(estimator=SVC(random_state=1234),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=2)

In [28]:
grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [29]:
predictions = grid.best_estimator_.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       532
           1       0.85      0.74      0.79        61

    accuracy                           0.96       593
   macro avg       0.91      0.86      0.88       593
weighted avg       0.96      0.96      0.96       593

